# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7ff043bb7490>
├── 'a' --> tensor([[ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068]])
└── 'x' --> <FastTreeValue 0x7ff043bb7130>
    └── 'c' --> tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                        [ 0.8241, -0.7503,  1.3152,  0.1934],
                        [ 0.8663, -0.4170, -0.3685, -0.5202]])

In [4]:
t.a

tensor([[ 1.1538,  0.8675,  1.5108],
        [-1.4594, -2.0900,  0.9068]])

In [5]:
%timeit t.a

60.9 ns ± 0.0455 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7ff043bb7490>
├── 'a' --> tensor([[ 0.1482,  0.0428,  0.7410],
│                   [ 0.1929, -0.2148, -0.5137]])
└── 'x' --> <FastTreeValue 0x7ff043bb7130>
    └── 'c' --> tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                        [ 0.8241, -0.7503,  1.3152,  0.1934],
                        [ 0.8663, -0.4170, -0.3685, -0.5202]])

In [7]:
%timeit t.a = new_value

63.1 ns ± 0.0205 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1538,  0.8675,  1.5108],
               [-1.4594, -2.0900,  0.9068]]),
    x: Batch(
           c: tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1538,  0.8675,  1.5108],
        [-1.4594, -2.0900,  0.9068]])

In [11]:
%timeit b.a

56.7 ns ± 0.013 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6971,  1.2187,  0.0995],
               [ 1.2982,  0.4368, -0.9995]]),
    x: Batch(
           c: tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.14 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

838 ns ± 0.0995 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 10 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.88 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 172 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fef89d92dc0>
├── 'a' --> tensor([[[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]],
│           
│                   [[ 1.1538,  0.8675,  1.5108],
│                    [-1.4594, -2.0900,  0.9068]]])
└── 'x' --> <FastTreeValue 0x7fef89d92a30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.3 µs ± 30.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fef89d92280>
├── 'a' --> tensor([[ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068],
│                   [ 1.1538,  0.8675,  1.5108],
│                   [-1.4594, -2.0900,  0.9068]])
└── 'x' --> <FastTreeValue 0x7fef89d92100>
    └── 'c' --> tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                        [ 0.8241, -0.7503,  1.3152,  0.1934],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 80.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.5 µs ± 87.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1628,  0.2781, -0.8571, -0.0814],
                       [ 0.8241, -0.7503,  1.3152,  0.1934],
                       [ 0.8663, -0.4170, -0.3685, -0.5202]],
              
                      [[-0.1628,  0.2781, -0.8571, -0.0814],
                       [ 0.8241, -0.7503,  1.3152,  0.1934],
                       [ 0.8663, -0.4170, -0.3685, -0.5202]],
              
                      [[-0.1628,  0.2781, -0.8571, -0.0814],
                       [ 0.8241, -0.7503,  1.3152,  0.1934],
                       [ 0.8663, -0.4170, -0.3685, -0.5202]],
              
                      [[-0.1628,  0.2781, -0.8571, -0.0814],
                       [ 0.8241, -0.7503,  1.3152,  0.1934],
                       [ 0.8663, -0.4170, -0.3685, -0.5202]],
              
                      [[-0.1628,  0.2781, -0.8571, -0.0814],
                       [ 0.8241, -0.7503,  1.3152,  0.1934],
                       [ 0.8663, -0.4170, -0.3685, -0.5202]],

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 254 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202],
                      [-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202],
                      [-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202],
                      [-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202],
                      [-0.1628,  0.2781, -0.8571, -0.0814],
                      [ 0.8241, -0.7503,  1.3152,  0.1934],
                      [ 0.8663, -0.4170, -0.3685, -0.5202],
                      [-0.1628,  0.2781, -0.8571, -0.0814],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 348 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 1.51 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
